In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
test = pd.read_csv("../input/machine-learning-course-project-cs-412/loansTest.csv")
train = pd.read_csv("../input/machine-learning-course-project-cs-412/loansTrain.csv")

In [ ]:
sampleSubmission = pd.read_csv("../input/machine-learning-course-project-cs-412/sampleSubmission.csv")

In [ ]:
sampleSubmission.head()

In [ ]:
test

In [ ]:
test.dtypes

In [ ]:
test.columns[1:]

In [ ]:
# extract feature columns
feature_cols = list(train.columns[:-1])
# extract target column 'loan_status'
target_col = train.columns[-1]

# separate the data into feature data and target data
X_all = train[feature_cols]
y_all = train[target_col]

# extract feature columns
feature_cols = list(test.columns[1:])
# extract target column 'loan_status'
target_col = test.columns[0]

# separate the data into feature data and target data
X_test = test[feature_cols]
id_test = test[target_col]

X_all = pd.concat([X_all, X_test], ignore_index=True, sort=False)

In [ ]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['36 months', '60 months'], [0, 1])
            col_data = col_data.replace([' 36 months', ' 60 months'], [0, 1])


        print(col)
        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:

#Use Scikit-Learn's train_test_split function to create train and validation sets.

# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)

In [ ]:
X_all['emp_title'].fillna("Unemployed",inplace=True)
# replace null with zeros
for col in X_all.columns:
    X_all[col].fillna(0,inplace=True) 

In [ ]:
# feature removel
X_all.drop(['zip_code', 'dti', 'avg_cur_bal', 'emp_title', 'delinq_amnt','percent_bc_gt_75','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','tax_liens'], axis=1, inplace=True)
# feature removel
# X_test.drop(['zip_code', 'dti', 'avg_cur_bal', 'emp_title'], axis=1, inplace=True)

In [ ]:
# X_all = pd.get_dummies(X_all)
X_all = preprocess_features(X_all)
X, X_test = X_all.iloc[:1000000,:], X_all.iloc[1000000:,:]
X_train, X_val, y_train, y_val = train_test_split(X, y_all, train_size=0.8)

In [ ]:
from time import time
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds",format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
train_predict(clf, X_train, y_train, X_val, y_val)

In [ ]:
y_pred = clf.predict(X_test)

#predict_labels(clf, X_test, y_test)

In [ ]:
len(y_pred)

In [ ]:
loan = pd.DataFrame({'loan_status': y_pred[:]})

In [ ]:
loan

In [ ]:
result = pd.concat([id_test,loan], axis=1, sort=False)

In [ ]:
result

In [ ]:
#result.to_csv('result.csv', index=False)